## 数据结构和算法

### 保留最后 N 个元素

`deque` 用于保留有限的历史纪录。它是一个双端队列，在两端插入的时间复杂度均为 `O(1)`。当不指定 `maxlen` 时其长度无限制。

In [5]:
from collections import deque

last_items = deque(maxlen=5)

for i in range(10):
    last_items.append(i)
    
last_items

deque([5, 6, 7, 8, 9])

### 查找最大或最小的 K 个元素

当需要返回一个列表中最大或最小的 K 个元素时，可以使用函数 `nlargest()` 和 `nsmallest()`。

如果 K 的大小和列表长度接近时候，堆就没有太大优势了，此时先排序再切片会更好。

In [10]:
import heapq
nums = [1, 8, 2, 23, 7, -44, 18, 23, 42, 37, 2]

heapq.nlargest(3, nums) # [42, 37, 23]
heapq.nsmallest(3, nums) # [-4, 1, 2]
heapq.nlargest(3, nums, key=lambda x: abs(x)) # [-44, 42, 37]

### 查找两字典的相同点

`dict` 的 `keys` 和 `items` 方法返回的对象也支持集合操作。比如要完成对键取交集或并集的操作，不需要先转成 `set`。

In [18]:
a = {'a': 1, 'b': 2}
b = {'a': 1, 'c': 3}
a.keys() | b.keys(), a.items() & b.items()

({'a', 'b', 'c'}, {('a', 1)})

### 同时打乱两个列表

有的时候两个列表中的元素对应下标是有关系的，因此在打乱的时候，需要把两个列表按照相同的方式打乱。

```python
name_age_list = list(zip(names, ages))
np.random.shuffle(name_age_list)
names[:], ages[:] = zip(*name_age_list)
```

### 命名切片

使用命名切片可以让代码更易读。

In [17]:
record = 'ID.1234-10.30'

ID = slice(0, 7)
DATE = slice(8, None)
record[ID], record[DATE]

('ID.1234', '10.30')

### 根据某一键的值来对字典列表排序

通常可以使用 `lambda` 表达式来获取某个键的值，不过使用 `itemgetter` 更为方便，对于对象可以使用 `attrgetter` 来获取属性。

```python
from operator import itemgetter

rows = [
    {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003},
    {'fname': 'Brian', 'lname': 'Beazley', 'uid': 1002}
]

rows.sort(key=itemgetter('fname', 'lname'))
```

### 使用生成器作为参数

有的函数需要参数支持迭代器接口，比如下面的例子，此时，直接传入生成器会更好。而如果转换为列表，就需要创建一个只会使用一次的列表。

```python
sum(n * n for n in nums)  # good
sum([n * n for n in nums]) # bad
```

### 在多个字典上查找

如果需要在多个 `dict` 中查找某个键，当然可以以此在各个 `dict` 中去查找。或者将多个 `dict` 合并成一个。不过使用 `ChainMap` 可以得到类似于变量的作用域链那种结构。而且被链接起来的多个 `dict` 的变化会得到反映。

```python
from collections import ChainMap
a = {'x': 1, 'z': 3}
b = {'x': 2, 'y': 2}
c = ChainMap(a, b)

print(c['x']) # Outputs 1 (from a)
print(c['y']) # Outputs 2 (from b)
a['y'] = '4'
print(c['y']) # Outputs 4 (from a)
```

---------------

## 字符串和文本

### 分割字符串

切分字符串可以使用 `str.split` 但是它常常不够灵活性，因为如果指定分隔符，则分隔符只能为固定的一个。如果不指定，则默认以空白符分隔。

```python
text = 'a,b,c,d'
text.split(',')
>>> ['a', 'b', 'c', 'd']

text = 'a b c d'
text.split()
>>> ['a', 'b', 'c', 'd']
```

使用 `re.split` 就会灵活很多。

```python
import re

text = 'a+b-c*d'
re.split(r'[-+*]', text)
>>> ['a', 'b', 'c', 'd']
```

## 日期和时间

### 基本的日期与时间转换

定义某个时间点，可以使用 `datatime`，定义时间偏移量，可以使用 `timedelta`。两个时间点相减可得到 `timedelta`，一个时间点加减一个 `timedelta` 可以得到一个新的时间点。

In [1]:
from datetime import datetime
from datetime import timedelta

start = datetime(2019, 7, 2)
offset = timedelta(days=12, hours=12)
start + offset

datetime.datetime(2019, 7, 14, 12, 0)

### 字符串与日期之间的转换

使用 `datetime.now` 可以得到当前时刻

In [8]:
datetime.now().strftime('%Y-%m-%d %H:%M:%S')

'2019-07-12 19:34:53'

In [11]:
text = '2019-07-12 19:34:53'
datetime.strptime(text, '%Y-%m-%d %H:%M:%S')

datetime.datetime(2019, 7, 12, 19, 34, 53)

## 迭代器与生成器

### 实现迭代器接口

一个对象为了实现迭代器接口，需要实现 `__iter__` 方法，此方法需要返回一个含有 `__next__` 方法的对象。通常在 `__next__` 方法需要在新的对象上实现，因为 `__next__` 方法需要用到一些状态变量，这需要存在一个对象中。为啥不能在当前对象上实现 `__next__` 呢，因为在对一个对象同时创建多个迭代器的时候，其中用到的状态就会冲突。

这里创建了一个 `Stack` 类用来演示迭代器接口。实际中，当然不必如此繁琐。

In [1]:
class Stack:
    def __init__(self):
        self.items = []

    def pop(self):
        return self.items.pop()

    def push(self, item):
        self.items.append(item)

    def __iter__(self):
        print('Stack.__iter__')
        return StackIterator(self.items)

    def __reversed__(self):
        print('Stack.__reversed__')
        return StackIterator(self.items, reverse=True)

class StackIterator:
    def __init__(self, items, reverse=False):
        self.items = items
        self.sp = len(items) - 1
        self.bp = 0
        self.reverse = reverse

    def __iter__(self):
        print('StackIterator.__iter__')
        return self
        
    def __next__(self):
        print('StackIterator.__next__')
        if self.reverse:
            return self.__last()
        else:
            return self.__next()
    
    def __next(self):
        if self.sp < 0:
            raise StopIteration()
        else:
            self.sp -= 1
            return self.items[self.sp + 1]

    def __last(self):
        if self.bp < len(self.items):
            self.bp += 1
            return self.items[self.bp - 1]
        else:
            raise StopIteration()

stack = Stack()
stack.push(1)
stack.push(2)
stack.push(3)

`iter` 函数会调用对象的 `__iter__` 方法以返回迭代器。用迭代器作为参数调用 `next` 方法，会代理到 `__next__` 方法上，每次调用都会返回一个元素，具体迭代策略由具体的类来决定。当迭代完了所有元素时，抛出 `StopIteration` 异常以表示迭代结束。

In [2]:
it = iter(stack)
while True:
    try:
        print(next(it))
    except StopIteration:
        break

Stack.__iter__
StackIterator.__next__
3
StackIterator.__next__
2
StackIterator.__next__
1
StackIterator.__next__


实现了迭代器接口的对象，可以使用 `list` 来将所有元素转换列表。`list` 会自动驱动迭代器。

In [3]:
list(stack)

Stack.__iter__
StackIterator.__next__
StackIterator.__next__
StackIterator.__next__
StackIterator.__next__


[3, 2, 1]

要想对一个对象实现反向迭代，可以实现 `__reversed__` 接口，此方法需要返回一个迭代器对象。`reversed` 函数会调用此方法。

有时候需要使用下面的写法返回一个反向的列表，此时需要注意的是，`list` 希望参数支持迭代器接口，即含有一个 `__iter__` 方法，并返回一个含有 `__next__` 方法的对象。因此返回的迭代器对象也要有 `__iter__` 方法，在此方法中直接返回自身就可以了。

In [8]:
list(reversed(stack))

Stack.__reversed__
StackIterator.__iter__
StackIterator.__next__
StackIterator.__next__
StackIterator.__next__
StackIterator.__next__


[1, 2, 3]

实际上只需要将接口代理到底层数据结构的对应接口上即可。

```python
def __iter__(self):
    return iter(self.items)

def __reversed__(self):
    return reversed(self.items)
```

### 对迭代器切片

对于没有实现 `__getitem__` 和 `__len__` 接口的对象是不能进行常规的切片操作的。迭代器没有办法用下标进行索引，且长度未知，常规的切片无法作用于迭代器上，不过可以使用 `itertools.islice` 来实现对迭代器的切片。

In [16]:
from itertools import islice

c = {1,2,3,4,5,6,7,8,9}

[x for x in islice(c, 0, 8, 2)]

[1, 3, 5, 7]

### 展开嵌套序列

In [21]:
from collections import Iterable

def flatten(items):
    for x in items:
        if isinstance(x, list):
            yield from flatten(x)
        else:
            yield x

items = [1, 2, [3, 4, [5, [6]], 7], 8]

list(flatten(items))

[1, 2, 3, 4, 5, 6, 7, 8]

`iter` 有一个特性，它接受一个函数作为参数，`iter` 会不断调用此函数直到输出结果等于第二个参数为止。

In [14]:
n = 0
def plus():
    global n
    n += 1
    return n

list(iter(plus, 5))

[1, 2, 3, 4]

## Python 语法细节

### `classmethod` 与 `staticmethod`

```python
class Demo:
    @classmethod
    def klass_method(*args):
        return args
    @staticmethod
    def static_method(*args):
        return args
```
    
`classmethod` 修饰的函数，其第一个参数为类本身而不是类的实例。

```python
>>> Demo.klass_method(1, 2, 3)
<<< (__main__.Demo, 1, 2, 3)
```

通常 `classmethod` 需要在函数中来创建类，且第一个参数习惯上命名为 `cls`。

```python
class Demo:
    def __init__(self, *args):
        pass
    @classmethod
    def klass_method(cls, *args):
        return cls(*args)
```
    
staticmethod 修饰的函数，就是普通的函数，只是挂着类的名下罢了。`staticmethod` 修饰的函数，可以看做以类为命名空间的一系列函数，比如 `math` 下面的函数。

```python
>>> Demo.static_method(1, 2, 3)
<<< (1, 2, 3)
```


### `global`

`global` 用在函数内部，用来指出某个变量在 global 命名空间下，即为全局变量。在 Python 中，读取一个变量时，会先在局部作用域下查找，如果找不到再向外层作用域逐层查找。在一个变量赋值的时候，如果在局部作用域下找不到该变量，就会在局部作用域下创建此变量。

如果在函数内部，希望对全局变量赋值，就需要明确指出该变量为全局变量，否则赋值的时候会创建新的局部变量。

```python
name = 'python'

def change_name():
    name = name.title()
    
change_name()
```

这段代码会出错，错误信息为 `UnboundLocalError: local variable 'name' referenced before assignment`，原因是函数 `change_name` 中，有对 `name` 的赋值操作，这会让 `name` 被视为局部变量，对未赋值的局部变量进行读取，就出了错误。


```python
def change_name():
    global name
    name = name.title()
```

使用关键字 `global` 来指明 `name` 为全局变量，就不会出错了。

### `nonlocal`

关键字 `nonlocal` 用在定义在函数体中的函数内。函数体内部定义的函数，可以使用外层作用域的变量。但依然存在 `global` 关键词要解决的那个问题。在对外部作用域的变量赋值的时候，此变量会被视为局部变量。

```python
def make_plus():
    total = 0
    
    def plus(n):
        nonlocal total
        total += n
        return total
    return plus

plus = make_plus()
plus(1)
```

上面这个函数，如果没有 `nonlocal total` 这一行。那么执行到 `total += n` 时报错，因为这一行等价于 `total = total + n`，而 `total` 是局部变量，在没有赋值的情况下读取 `total` 会报错。`nonlocal` 关键词用来说明变量不是局部变量，需要在外层作用域去找。

之所以需要 `global` 和 `nonlocal` 这两个关键词，是因为 Python 中声明变量和对变量赋值代码层面是一样的。而在其他语言中，声明变量会使用一些关键词，比如下面这样的：


```js
let name = '11'
var n = 1
int m = 2
```

如此，就可以区分开声明变量和对现有变量赋值，这两种操作了。Python 抛掉了这些，因此需要增加两个关键字来挽救。

## 文件操作

对文件每一行进行迭代，可以直接在文件上进行迭代，因为文件迭代器对文件内容按行迭代。

```python
with open('./list.txt', encoding='utf-8', mode='r') as fin:
    for line in fin:
        print(line) # 行尾含 `\n`
```

readline 每次读取一行，如果没有内容了，会返回空行

```python
with open('./list.txt', encoding='utf-8', mode='r') as fin:
    line = fin.readline()
    while line:
        print(line)
        line = fin.readline()  # 行尾含 `\n`
    print(len(line))
```
        
`read` 可以带有参数，参数指定读取的字符数量

```python
with open('./list.txt', encoding='utf-8', mode='r') as fin:
    char = fin.read(1)
    while char:
        print(char)
        char = fin.read(1)
```


如果不带参数，会一次性读取所有内容

```python
with open('./list.txt', encoding='utf-8', mode='r') as fin:
    content = fin.read()
    print(content)
```

`readlines` 返回一个由每一行构成的数组

```python
with open('./list.txt', encoding='utf-8', mode='r') as fin:
    lines = fin.readlines()
    print(lines)
```


**文件中某些行有编码错误**

我在使用 pandas 读取 csv 文件时，pandas 报了如下错误：

```
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xed in position 3: invalid continuation byte
```

其原因是文件中某几行有编码问题，至于为什么，不得而知。解决方法是移除掉出错的行。

```python
def remove_bad_lines(in_file, out_file):
    fin = open(in_file, mode='r', encoding='utf-8', errors='ignore')
    fout = open(out_file, mode='w', encoding='utf-8')
    for line in fin:
        fout.write(line)
    fin.close()
    fout.close()
```

`errors='ignore'` 会忽略掉所有存在编码错误的行。

## 多线程/多进程

### ProcessPoolExecutor


```python
from concurrent.futures import ProcessPoolExecutor

def run(items, worker, n_works=12):
    groups = []
    executor = ProcessPoolExecutor(max_workers=n_works)
    group_size = int(len(items) / n_works + 1)
    
    for i in range(n_works):
        start = i * group_size
        end = start + group_size
        groups.append(items[start: end])
        
    results_list = executor.map(worker, groups)
    results_list = list(results_list)
    
    all_results = []
    for results in results_list:
        all_results.extend(results)

    return all_results

labels = run(samples, clf.predict)
```